# Chapter 4 - Scene Construction
 

In [ ]:
from manim import *

Scenes in manim can become cluttered very easily. As you want to have videos in the end, and not the code, code tidyness is not crucial, but often it is useful to stay organized and to not get lost in complex scenes.